<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%94%D0%9E%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Динамические окрестностные модели (ДОМ)
####Вариант 5

In [133]:
import numpy as np
import pandas as pd
import random
import copy
import math
from prettytable import PrettyTable
import itertools as it

In [134]:
#A={a1, a2, a3}

#какие дуги входят в позицию:
#Ox[a1]={a1}
#Ox[a2]={}
#Ox[a3]={a1, a2}

#в какие позиции входят v
#Ov[a1]={a1}
#Ov[a2]={a2}
#Ov[a3]={a3}

#количества состояний/входов/выходов
elem = 100
count = [3, 3, 3]
Sx = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])
Sv = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

print("Матрица смежности:")
print("Sx = ")
print(Sx)
print("Sv = ")
print(Sv)

Матрица смежности:
Sx = 
[[1 0 1]
 [0 0 1]
 [0 0 0]]
Sv = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [135]:
def x1(x, v):
  return math.sin(math.log(abs(v[1] + v[0])) / x[2])

def x2(x, v):
  return math.cos(v[0] * x[2] - (x[1] / x[2])**3)

def x3(x, v):
  return math.sin(math.tan(v[0] / v[2]) - 3**(x[2]))

def v1(x, v):
  return (x[1] * v[0] * v[1] + math.tan(x[2])**2)

def v2(x, v):
  return (0.2 * v[0] + 33.0 * v[1] + v[2])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * x[2])))

def y1(x, v):
  return (x[0] / v[0]) * math.sqrt(abs(x[0] / x[1]))

def y2(x, v):
  return math.sqrt(abs(math.sin(v[2])))

def y3(x, v):
  return v[2] * v[0] / v[0]

functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]

In [138]:
#Метод псевдообращения Бена-Израиля
def BenIsrael(A, eps = 0.000001): #А - np.array, eps - точность вычислительная
  I = np.diag(np.diag(np.ones([len(A),len(A)])))
  buf = 1
  i = 1

  if(np.linalg.norm(A, ord='fro') != 0): #если не нулевая норма
    gamma = 1.6 / (np.linalg.norm(A, ord='fro')**2)
    A1 = gamma * (A.T)
    i += 1
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
  else: #если нулевая норма
    A1 = np.zeros((len(A[0]), len(A)))
    buf = 0

  while(E >= eps and buf):
    A1 = A2.copy()
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
    i += 1

  return A1

#решение СЛАУ вида AX=B
def solvingSLAE(A, B):
  A = np.array(A.copy())

  #проверка на бесконечное количество решений
  A_psev = BenIsrael(A)

  X = A_psev.dot(B)
  return X

#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, x, v, typeFunc = 1, elem = 100):
  vector = []

  if(typeFunc == 0):
    for i in range(elem):
      vector.append(function(generationRandomVector(), generationRandomVector()))
  else:
    for i in range(elem):
      vector.append(function(x[i], v[i]))

  return vector

#=================================================================
#Создание ДОМ в определённый момент времени
def createTableDNM(functions, count, step, oldTable = []): #count(x, v, y)
  table = []
  vector = []
  columns = createColumns(step, count)

  if(step == 0):
    for i in range(sum(count[0:2])):
      table.append(createVariants(functions[i], [], [], 0)) #xi, vi

  else:
    for i in range(count[0]): #xi
      table.append(createVariants(functions[i], (np.array(oldTable))[:, 0:count[0]], (np.array(oldTable))[:, count[0]:sum(count[0:2])]))
    for i in range(count[1]): #vi
      table.append(createVariants(functions[count[0] + i], [], [], 0))

  for i in range(count[2]): #yi
    table.append(createVariants(functions[sum(count[0:2]) + i], (np.array(table).T)[:, 0:count[0]], (np.array(table).T)[:, count[0]:sum(count[0:2])]))

  table = pd.DataFrame(np.array(table).T, columns = columns)
  return table

#заполнение
def createColumns(step, count):
  columns = []
  for i in range(count[0]):
    columns.append("x" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[1]):
    columns.append("v" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[2]):
    columns.append("y" + str(i + 1) + "(" + str(step) + ")")
  return columns

In [139]:
#functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]
model_0 = createTableDNM(functions, count, 0)
model_1 = createTableDNM(functions, count, 1, np.array(model_0))
model_2 = createTableDNM(functions, count, 2, np.array(model_1))
model_3 = createTableDNM(functions, count, 3, np.array(model_2))
model_4 = createTableDNM(functions, count, 4, np.array(model_3))
model_5 = createTableDNM(functions, count, 5, np.array(model_4))
model_6 = createTableDNM(functions, count, 6, np.array(model_5))
model_7 = createTableDNM(functions, count, 7, np.array(model_6))
model_8 = createTableDNM(functions, count, 8, np.array(model_7))
model_9 = createTableDNM(functions, count, 9, np.array(model_8))
model_10 = createTableDNM(functions, count, 10, np.array(model_9))

In [140]:
model_9

,x1(9),x2(9),x3(9),v1(9),v2(9),v3(9),y1(9),y2(9),y3(9)
0,-0.232242,0.936033,-0.310279,0.620523,86.504338,0.954550,-0.186427,0.903357,0.954550
1,0.217579,-0.947651,-0.989302,8.655621,97.479683,1.175445,0.012045,0.960657,1.175445
2,0.435019,0.950957,-0.365278,-16.380707,-67.793787,1.066639,-0.017962,0.935725,1.066639
3,0.983793,-0.636787,-0.437446,3.576442,-24.970520,10.831166,0.341906,0.993235,10.831166
4,0.857292,-0.924242,-0.046113,658.290562,6.934467,0.953319,0.001254,0.902963,0.953319
...,...,...,...,...,...,...,...,...,...
95,0.189294,0.880653,-0.139243,0.249884,-58.650772,1.004094,0.351206,0.918518,1.004094
96,-0.910998,0.648409,0.993894,0.085195,-7.675053,0.791112,-12.674646,0.843288,0.791112
97,0.405297,0.823365,0.751502,0.342603,-6.725258,0.047556,0.829990,0.218033,0.047556
98,0.958095,-0.884842,-0.436509,11.964935,71.871231,2.567564,0.083324,0.736898,2.567564


In [141]:
model_10

,x1(10),x2(10),x3(10),v1(10),v2(10),v3(10),y1(10),y2(10),y3(10)
0,-0.966221,-0.530030,0.049146,-1.459179,-72.112704,5.259570,0.894038,0.924118,5.259570
1,0.999996,-0.999852,0.999403,2.415573,77.041550,4.797820,0.414009,0.998175,4.797820
2,0.417520,0.066098,-0.299035,5.629659,1.085912,2.210872,0.186397,0.895573,2.210872
3,-0.658696,-0.063173,-0.272197,7.687714,52.945143,0.342878,-0.276672,0.579827,0.342878
4,-0.399923,-0.190001,-0.994863,-3.095940,72.523465,1.263249,0.187411,0.976258,1.263249
...,...,...,...,...,...,...,...,...,...
95,0.805217,-0.053077,-0.567954,3.862982,-63.129169,6.844621,0.811879,0.729659,6.844621
96,0.892261,0.981434,-0.266499,16.120334,-76.337480,8.575236,0.052776,0.866590,8.575236
97,0.624964,0.490861,-0.823385,-5.685336,55.202872,14.826759,-0.124036,0.878354,14.826759
98,0.660373,0.552128,-0.376304,-1.602385,75.349616,1.488063,-0.450710,0.998288,1.488063


##1. Линейная форма функции пересчёта

In [191]:
def buildLinearRegretion(model1, model2, cols):
  vector_ones = [1] * elem
  coef = []

  linearFunction = model1[cols[0:(len(cols) - 1)]]
  linearFunction[cols[len(cols) - 1]] = model2[cols[len(cols) - 1]]
  linearFunction.insert(0, "optional_col", vector_ones)

  A_linear = np.array(linearFunction)
  #A_psev = BenIsrael(A_linear[:, 0:(len(A_linear[0]) - 1)])
  X_linear = solvingSLAE(A_linear[:, 0:(len(A_linear[0]) - 1)], np.matrix(A_linear[:, len(A_linear[0]) - 1]).T)

  estimate_linear = []
  variance_linear = []
  error_linear = []

  for i in range(len(linearFunction)):
    for j in range(0, len(X_linear)):
      if(j == 0):
        estimate_linear.append(X1_linear[j,0])
      else:
        estimate_linear[i] += X_linear[j,0] * np.array(linearFunction)[i,j]

    variance_linear.append((1.0 / len(linearFunction)) * (np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])**2)
    error_linear.append((1.0 / len(linearFunction)) * abs((np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])
                            / np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1]) * 100)

  linearFunction['estimate'] = estimate_linear
  linearFunction['variance'] = variance_linear
  linearFunction['error, %'] = error_linear

  return X_linear, linearFunction

###Для функции $x_1[t+1]$

In [192]:
#x3, v1, v2
X1_linear, X1_linearTable = buildLinearRegretion(model_9, model_10, ['x3(9)', 'v1(9)', 'v2(9)', 'x1(10)'])
print(pd.DataFrame(X1_linear, columns=['X1'], index=['a1', 'c13', 'b11', 'b12']))
X1_linearTable

           X1
a1   0.108018
c13 -0.420932
b11 -0.000014
b12 -0.000141


<ipython-input-191-4ded4159bb3a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linearFunction[cols[len(cols) - 1]] = model2[cols[len(cols) - 1]]
<ipython-input-191-4ded4159bb3a>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linearFunction['estimate'] = estimate_linear
<ipython-input-191-4ded4159bb3a>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

,optional_col,x3(9),v1(9),v2(9),x1(10),estimate,variance,"error, %"
0,1,-0.310279,0.620523,86.504338,-0.966221,0.226451,0.014225,1.234368
1,1,-0.989302,8.655621,97.479683,0.999996,0.510618,0.002395,0.489380
2,1,-0.365278,-16.380707,-67.793787,0.417520,0.271536,0.000213,0.349646
3,1,-0.437446,3.576442,-24.970520,-0.658696,0.295614,0.009107,1.448787
4,1,-0.046113,658.290562,6.934467,-0.399923,0.117304,0.002675,1.293317
...,...,...,...,...,...,...,...,...
95,1,-0.139243,0.249884,-58.650772,0.805217,0.174873,0.003973,0.782824
96,1,0.993894,0.085195,-7.675053,0.892261,-0.309265,0.014437,1.346609
97,1,0.751502,0.342603,-6.725258,0.624964,-0.207372,0.006928,1.331814
98,1,-0.436509,11.964935,71.871231,0.660373,0.281485,0.001436,0.573748


In [193]:
print("E = " + str(sum(X1_linearTable['variance'])))
print("A = " + str(sum(X1_linearTable['error, %'])))

E = 0.4327245892011757
A = 100.22978781799036


###Для функции $x_2[t+1]$

In [194]:
#x2, x3, v1
X2_linear, X2_linearTable = buildLinearRegretion(model_9, model_10, ['x2(9)', 'x3(9)', 'v1(9)', 'x2(10)'])
print(pd.DataFrame(X2_linear, columns=['X2'], index=['a2', 'c22', 'c23', 'b21']))
X2_linearTable

           X2
a2   0.079645
c22 -0.007670
c23  0.087813
b21 -0.000014


<ipython-input-191-4ded4159bb3a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linearFunction[cols[len(cols) - 1]] = model2[cols[len(cols) - 1]]
<ipython-input-191-4ded4159bb3a>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linearFunction['estimate'] = estimate_linear
<ipython-input-191-4ded4159bb3a>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

,optional_col,x2(9),x3(9),v1(9),x2(10),estimate,variance,"error, %"
0,1,0.936033,-0.310279,0.620523,-0.530030,0.073583,3.643484e-03,1.138828
1,1,-0.947651,-0.989302,8.655621,-0.999852,0.028289,1.057075e-02,1.028293
2,1,0.950957,-0.365278,-16.380707,0.066098,0.068881,7.742004e-08,0.042096
3,1,-0.636787,-0.437446,3.576442,-0.063173,0.074437,1.893659e-04,2.178316
4,1,-0.924242,-0.046113,658.290562,-0.190001,0.101699,8.508881e-04,1.535254
...,...,...,...,...,...,...,...,...
95,1,0.880653,-0.139243,0.249884,-0.053077,0.089032,2.019520e-04,2.677407
96,1,0.648409,0.993894,0.085195,0.981434,0.190320,6.258613e-03,0.806079
97,1,0.823365,0.751502,0.342603,0.490861,0.167690,1.044397e-03,0.658377
98,1,-0.884842,-0.436509,11.964935,0.552128,0.076303,2.264093e-03,0.861802


In [195]:
print("E = " + str(sum(X2_linearTable['variance'])))
print("A = " + str(sum(X2_linearTable['error, %'])))

E = 0.478813387541212
A = 104.70250266605706


###Для функции $x_3[t+1]$

In [196]:
#x3, v1, v3
X3_linear, X3_linearTable = buildLinearRegretion(model_9, model_10, ['x3(9)', 'v1(9)', 'v3(9)', 'x3(10)'])
print(pd.DataFrame(X3_linear, columns=['X3'], index=['a3', 'c33', 'b31', 'b33']))
X3_linearTable

           X3
a3  -0.124717
c33 -0.159815
b31 -0.000007
b33 -0.023512


<ipython-input-191-4ded4159bb3a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linearFunction[cols[len(cols) - 1]] = model2[cols[len(cols) - 1]]
<ipython-input-191-4ded4159bb3a>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linearFunction['estimate'] = estimate_linear
<ipython-input-191-4ded4159bb3a>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

,optional_col,x3(9),v1(9),v3(9),x3(10),estimate,variance,"error, %"
0,1,-0.310279,0.620523,0.954550,0.049146,0.135157,0.000074,1.750119
1,1,-0.989302,8.655621,1.175445,0.999403,0.238428,0.005791,0.761430
2,1,-0.365278,-16.380707,1.066639,-0.299035,0.141427,0.001940,1.472944
3,1,-0.437446,3.576442,10.831166,-0.272197,-0.076760,0.000382,0.717997
4,1,-0.046113,658.290562,0.953319,-0.994863,0.088512,0.011737,1.088969
...,...,...,...,...,...,...,...,...
95,1,-0.139243,0.249884,1.004094,-0.567954,0.106661,0.004551,1.187798
96,1,0.993894,0.085195,0.791112,-0.266499,-0.069423,0.000388,0.739499
97,1,0.751502,0.342603,0.047556,-0.823385,-0.013204,0.006564,0.983963
98,1,-0.436509,11.964935,2.567564,-0.376304,0.117329,0.002437,1.311792


In [197]:
print("E = " + str(sum(X3_linearTable['variance'])))
print("A = " + str(sum(X3_linearTable['error, %'])))

E = 0.41644092456615917
A = 147.72644580882027


###Для функции $y_1[t]$

In [198]:
#x1, x2, v1
Y1_linear, Y1_linearTable = buildLinearRegretion(model_9, model_9, ['x1(9)', 'x2(9)', 'v1(9)', 'y1(9)'])
print(pd.DataFrame(Y1_linear, columns=['Y1'], index=['a1', 'c11', 'c12', 'b11']))
Y1_linearTable

           Y1
a1  -0.114184
c11  0.357301
c12 -0.185441
b11  0.000008


<ipython-input-191-4ded4159bb3a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linearFunction[cols[len(cols) - 1]] = model2[cols[len(cols) - 1]]
<ipython-input-191-4ded4159bb3a>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linearFunction['estimate'] = estimate_linear
<ipython-input-191-4ded4159bb3a>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

,optional_col,x1(9),x2(9),v1(9),y1(9),estimate,variance,"error, %"
0,1,-0.232242,0.936033,0.620523,-0.186427,-0.148537,0.000014,0.203244
1,1,0.217579,-0.947651,8.655621,0.012045,0.361566,0.001222,29.018091
2,1,0.435019,0.950957,-16.380707,-0.017962,0.086966,0.000110,5.841698
3,1,0.983793,-0.636787,3.576442,0.341906,0.577645,0.000556,0.689482
4,1,0.857292,-0.924242,658.290562,0.001254,0.591281,0.003481,470.424133
...,...,...,...,...,...,...,...,...
95,1,0.189294,0.880653,0.249884,0.351206,0.012345,0.001148,0.964849
96,1,-0.910998,0.648409,0.085195,-12.674646,-0.337724,1.521996,0.973354
97,1,0.405297,0.823365,0.342603,0.829990,0.100148,0.005327,0.879338
98,1,0.958095,-0.884842,11.964935,0.083324,0.614533,0.002822,6.375235


In [199]:
print("E = " + str(sum(Y1_linearTable['variance'])))
print("A = " + str(sum(Y1_linearTable['error, %'])))

E = 15.628978190128885
A = 23017.892513313607


###Для функции $y_2[t]$

In [201]:
#v3
Y2_linear, Y2_linearTable = buildLinearRegretion(model_9, model_9, ['v3(9)', 'y2(9)'])
print(pd.DataFrame(Y2_linear, columns=['Y2'], index=['a2', 'b23']))
Y2_linearTable

           Y2
a2   0.792862
b23 -0.004633


<ipython-input-191-4ded4159bb3a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linearFunction[cols[len(cols) - 1]] = model2[cols[len(cols) - 1]]
<ipython-input-191-4ded4159bb3a>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linearFunction['estimate'] = estimate_linear
<ipython-input-191-4ded4159bb3a>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

,optional_col,v3(9),y2(9),estimate,variance,"error, %"
0,1,0.954550,0.903357,0.103596,0.006396,0.885322
1,1,1.175445,0.960657,0.102572,0.007363,0.893227
2,1,1.066639,0.935725,0.103076,0.006933,0.889843
3,1,10.831166,0.993235,0.057839,0.008750,0.941767
4,1,0.953319,0.902963,0.103601,0.006390,0.885265
...,...,...,...,...,...,...
95,1,1.004094,0.918518,0.103366,0.006645,0.887464
96,1,0.791112,0.843288,0.104353,0.005460,0.876255
97,1,0.047556,0.218033,0.107797,0.000122,0.505591
98,1,2.567564,0.736898,0.096123,0.004106,0.869558


In [202]:
print("E = " + str(sum(Y2_linearTable['variance'])))
print("A = " + str(sum(Y2_linearTable['error, %'])))

E = 0.5255318748741902
A = 86.93590092376519


###Для функции $y_3[t]$

In [203]:
#v1, v3
Y3_linear, Y3_linearTable = buildLinearRegretion(model_9, model_9, ['v1(9)', 'v3(9)', 'y3(9)'])
print(pd.DataFrame(Y3_linear, columns=['Y3'], index=['a3', 'b31', 'b33']))
Y3_linearTable

               Y3
a3   5.637713e-12
b31 -2.378706e-16
b33  1.000000e+00


<ipython-input-191-4ded4159bb3a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linearFunction[cols[len(cols) - 1]] = model2[cols[len(cols) - 1]]
<ipython-input-191-4ded4159bb3a>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linearFunction['estimate'] = estimate_linear
<ipython-input-191-4ded4159bb3a>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

,optional_col,v1(9),v3(9),y3(9),estimate,variance,"error, %"
0,1,0.620523,0.954550,0.954550,1.062567,0.000117,0.113161
1,1,8.655621,1.175445,1.175445,1.283463,0.000117,0.091895
2,1,-16.380707,1.066639,1.066639,1.174657,0.000117,0.101269
3,1,3.576442,10.831166,10.831166,10.939184,0.000117,0.009973
4,1,658.290562,0.953319,0.953319,1.061336,0.000117,0.113307
...,...,...,...,...,...,...,...
95,1,0.249884,1.004094,1.004094,1.112112,0.000117,0.107577
96,1,0.085195,0.791112,0.791112,0.899130,0.000117,0.136539
97,1,0.342603,0.047556,0.047556,0.155574,0.000117,2.271370
98,1,11.964935,2.567564,2.567564,2.675582,0.000117,0.042070


In [204]:
print("E = " + str(sum(Y3_linearTable['variance'])))
print("A = " + str(sum(Y3_linearTable['error, %'])))

E = 0.011667845789503507
A = 66.79897446213069
